##Set Up

In [ ]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests

gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = r"F:/GIS/GIS_DATA/Monitoring/"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = r"C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

# local variables
fdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
## Final feature class to append to in Enterprise Geodatabase double check this
sdeSEZ = os.path.join(sdeBase, "sde.SDE.Monitoring\sde.SDE.SEZ_Assessment_Unit")

## GDB with Raw Data straight from S123
headcutgdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2023.gdb")
erosiongdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
invasiveplantgdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2023.gdb")
channelincisiongdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")

#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe... could just be on SDE
FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

### Stream Erosion 

In [ ]:
# Stream Erosion Data Processing from GDB

# Set the workspace to your Geodatabase
arcpy.env.workspace = erosiongdb

# Input feature class or table
input_fc = 'Stream_Erosion'

# Convert feature class to Pandas DataFrame
erosionfields = ['Assessment_Unit_Name', 'Shape_Length', 'Bank_Type', 'Percent_Unstable',]
erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(input_fc, erosionfields), columns=erosionfields)

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))

# Calculate the total length of banks assessed
total_banks_assessed = (erosiondf['Shape_Length'].sum()) * 2

# Calculate the total length of eroded banks
total_eroded_banks = (erosiondf['Shape_Length'] * erosiondf['bank_multiplier']).sum()

# Update the 'percent_unstable' column
erosiondf['percent_unstable'] = (total_eroded_banks / total_banks_assessed) * 100

print("Total Banks Assessed:", total_banks_assessed)
print("Total Eroded Banks:", total_eroded_banks)
print("Updated DataFrame:")
print(erosiondf)

# Write the DataFrame back to the feature class
arcpy.da.UpdateCursor(input_fc, erosionfields) as cursor:
    for index, row in erosiondf.iterrows():
        cursor.updateRow(row)


## Code for Grading each parameter

In [ ]:
# Grading for each parameter add to final SDE file?
#Defining Grade for Channel Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if 0 <= value < 5:
        return 'A'
    elif 5 <= value < 20:
        return 'B'
    elif 20 <= value < 50:
        return 'C'
    else:
        return 'D'


#Define Grade for Incision based off of bankful height to bankful depth

def categorize_incision(bankful_ratio)
     if 0 <= bankful_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankful_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankful_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Headcut based off of headcut size---fix this
def categorize_headcut(value)
     if value = 0
        return 'A'
    elif 1.2 <= value < 1.6:
        return 'B'
    elif value =
        return 'C'
    else:
        return 'D'

Define Grade for Invasive based off of invasive


In [ ]:
#Channel Incision Process Data


In [ ]:
#Invasive Species